Section D)

Write a Python program to compute the value of an American styled option written on non-dividend paying stock. Recall that an American option is an option that can be exercised any time before maturity.

Use the Monte Carlo methodology for pricing the option. Assume required data. 

Pricing American Options:
- Analytical Approximation
- Binomial Tree
- Monte Carlo

An exact analytical solution exists for European options, such as BS.  

For American options, however, we have to use numerical methods.
- Binomial Tree (i.e. Lattice) or approximations:
    
    - an accurate one
    
    - main drawback: slow and complex maybe. implementation of a more complex option payoff is difficult, especially when the payoff is path-dependent. For example, for an American double-average option with periodic sampling time points, the strike price is not known at the start of the option.  It can only be determined in the future and is therefore path-dependent.  Another example is an American forward start option. These options cannot be valued using the binomial tree approach.

- Barone-Andesi-Whaley Analytical Approx with BS:
    
    - Consequently, several researchers have developed approximate solutions that are faster. In this example we’re going to use the Barone-Andesi-Whaley approximation, which takes the value computed by the Black-Scholes Model and adds the value of the early exercise option that is available on American option. The Barone-Adesi and Whaley Model has the advantages of being fast, accurate and inexpensive to use. It is most accurate for options that will expire in less than one year.

- Monte Carlo simulation:
    - This method will allow us to implement more complex option payoffs with greater flexibility, even if the payoffs are path-dependent. 
    
    - Specifically, we use the Least-Squares Method of Longstaff and Schwartz in order to take into account the early exercise feature.  
    - The stock price is assumed to follow the Geometrical Brownian Motion and the dividend is simulated continuously.

    - IDEA:
    
    Using this approach, it would be optimal to exercise the option if the immediate payment is larger than the expected future cash flows, otherwise it should be kept.  
    
    Specifically, for each generated path, we regress the future payoffs on the basis functions of S and S2. The regression equation provides us with estimation for the expected value of future payoffs as a function of S and S2. This expected value is the value of holding on to the option, i.e. the continuation value. Using the regression equation, we can decide if it is preferable to exercise the option immediately or to wait one more period. This procedure is repeated backward from the maturity date to the time zero. Finally, the price of the option is calculated as the average value of all the discounted payoffs.

In [2]:
# valuing an American option
# !pip install QuantLib
from QuantLib import *

valuation_date = Date(22, 8, 2018)
Settings.instance().evaluationDate = valuation_date+2

calendar = Canada()
volatility = 42.66/100
day_count = Actual365Fixed()

underlying = 13.5
risk_free_rate = 2.13/100
dividend_rate = 1.2/100

exercise_date = Date(22, 8, 2021)
strike = 13
option_type = Option.Put

payoff = PlainVanillaPayoff(option_type, strike)
exercise = EuropeanExercise(exercise_date)
european_option = VanillaOption(payoff, exercise)

spot_handle = QuoteHandle(SimpleQuote(underlying))

flat_ts = YieldTermStructureHandle(FlatForward(valuation_date,risk_free_rate,day_count))
dividend_yield = YieldTermStructureHandle(FlatForward(valuation_date,dividend_rate,day_count))
flat_vol_ts = BlackVolTermStructureHandle(BlackConstantVol(valuation_date,calendar,volatility,day_count))
bsm_process = BlackScholesMertonProcess(spot_handle,dividend_yield,flat_ts,flat_vol_ts)

# European option
european_option.setPricingEngine(AnalyticEuropeanEngine(bsm_process))
bs_price = european_option.NPV()
print("European option price is ", bs_price)


# American option MC
MC_engine = MCAmericanEngine(bsm_process, 'PseudoRandom', timeSteps=50, polynomOrder=5, seedCalibration=42, requiredSamples=10 ** 5)
am_exercise = AmericanExercise(valuation_date, exercise_date)
MCamerican_option = VanillaOption(payoff, am_exercise)
MCamerican_option.setPricingEngine(MC_engine)
MC_price = MCamerican_option.NPV()
print("MC American option price is ", MC_price)

European option price is  3.235252138808746
MC American option price is  3.2819352096497094
